In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

In [5]:
data=pd.read_csv('./test.csv')

def transform(data):
    data.Loan_Status=data.Loan_Status.map({'N':0,'Y':1})
    data.Gender=data.Gender.map({'Male':1,'Female':2})
    data.Married=data.Married.map({'Yes':1,'No':0})
    data.Education=data.Education.map({'Graduate':1,'Not Graduate':0})
    data.Self_Employed=data.Self_Employed.map({'No':0,'yes':1})
    data.Dependents=data.Dependents.map({'3+':3,'1':1,'2':2,'0':0})
    data.Property_Area=data.Property_Area.map({'Urban':1,'Semiurban':2,'Rural':3})
    return data

data=transform(data)

def myimputer(data):
    columns=data.columns
    imp=Imputer(missing_values='NaN',strategy='most_frequent',axis=0)
    data=imp.fit_transform(data)
    data=pd.DataFrame(data,columns=columns)
    return data

data=data.drop(['Loan_ID'],axis=1)
data=myimputer(data)
data['income']=data['ApplicantIncome']+data['CoapplicantIncome']

data['income']=data['ApplicantIncome']+data['CoapplicantIncome']


data=data.drop(['ApplicantIncome','CoapplicantIncome'],axis=1)

In [7]:
predictor=[x for x in data.columns if x not in ['Loan_ID','Loan_Status']]

In [11]:
xgb1 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'rank:pairwise', nthread=4, scale_pos_weight=1, seed=27)
x_train,x_test,y_train,y_test=train_test_split(data[predictor],data['Loan_Status'])
model=xgb1.fit(x_train,y_train)
result=model.predict(x_test)
accuracy_score(result,y_test)
cross_val_score(xgb1,data[predictor],data['Loan_Status'],scoring='accuracy').mean()

0.76713693607524303

In [12]:
param_test1 = {'max_depth':range(3,10,2), 'min_child_weight':range(1,6,2) }
gsearch=GridSearchCV(estimator=xgb1,param_grid=param_test1,scoring='accuracy')
gsearch.fit(x_train,y_train)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

([mean: 0.78696, std: 0.02204, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.77826, std: 0.00959, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.76957, std: 0.01268, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.78261, std: 0.02412, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.77391, std: 0.01645, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.77391, std: 0.02497, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.78478, std: 0.01909, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.78043, std: 0.02167, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.77826, std: 0.02804, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.78261, std: 0.02412, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.77826, std: 0.02136, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.78261, std: 0.03112, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 1

In [23]:
xgb1 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=3, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'rank:pairwise', nthread=4, scale_pos_weight=1, seed=27)
cross_val_score(xgb1,data[predictor],data['Loan_Status'],scoring='accuracy').mean()

0.77038099792762627

In [18]:
param_test2b = {'min_child_weight':[6,8,10,12]}
gsearch=GridSearchCV(estimator=XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'rank:pairwise',
 nthread=4,
 scale_pos_weight=1,
 seed=27),param_grid=param_test2b,scoring='accuracy')
gsearch.fit(x_train,y_train)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

([mean: 0.78478, std: 0.02362, params: {'min_child_weight': 6},
  mean: 0.78043, std: 0.02059, params: {'min_child_weight': 8},
  mean: 0.77609, std: 0.02189, params: {'min_child_weight': 10},
  mean: 0.76957, std: 0.02660, params: {'min_child_weight': 12}],
 {'min_child_weight': 6},
 0.7847826086956522)

In [23]:
xgb1 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'rank:paiwise', nthread=4, scale_pos_weight=1, seed=27)
model=xgb1.fit(x_train,y_train)
result=model.predict(x_test)
accuracy_score(result,y_test)

0.74675324675324672

In [25]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch=GridSearchCV(estimator=XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'rank:pairwise',
 nthread=4,
 scale_pos_weight=1,
 seed=27),param_grid=param_test3,scoring='accuracy')
gsearch.fit(x_train,y_train)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

([mean: 0.78696, std: 0.02204, params: {'gamma': 0.0},
  mean: 0.78478, std: 0.02136, params: {'gamma': 0.1},
  mean: 0.79130, std: 0.02351, params: {'gamma': 0.2},
  mean: 0.78913, std: 0.01619, params: {'gamma': 0.3},
  mean: 0.79130, std: 0.01940, params: {'gamma': 0.4}],
 {'gamma': 0.2},
 0.79130434782608694)

In [31]:
xgb1 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8, objective= 'rank:pairwise', nthread=4, scale_pos_weight=1, seed=27)
cross_val_score(xgb1,data[predictor],data['Loan_Status'],scoring='accuracy').mean()

0.77036505659174237

In [32]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch=GridSearchCV(estimator=XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'rank:pairwise',
 nthread=4,
 scale_pos_weight=1,
 seed=27),param_grid=param_test4,scoring='accuracy')
gsearch.fit(x_train,y_train)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

([mean: 0.77174, std: 0.01908, params: {'subsample': 0.6, 'colsample_bytree': 0.6},
  mean: 0.78478, std: 0.01391, params: {'subsample': 0.7, 'colsample_bytree': 0.6},
  mean: 0.78478, std: 0.02299, params: {'subsample': 0.8, 'colsample_bytree': 0.6},
  mean: 0.79130, std: 0.02136, params: {'subsample': 0.9, 'colsample_bytree': 0.6},
  mean: 0.77609, std: 0.01062, params: {'subsample': 0.6, 'colsample_bytree': 0.7},
  mean: 0.76739, std: 0.02168, params: {'subsample': 0.7, 'colsample_bytree': 0.7},
  mean: 0.78696, std: 0.01870, params: {'subsample': 0.8, 'colsample_bytree': 0.7},
  mean: 0.78913, std: 0.01619, params: {'subsample': 0.9, 'colsample_bytree': 0.7},
  mean: 0.78043, std: 0.01561, params: {'subsample': 0.6, 'colsample_bytree': 0.8},
  mean: 0.77391, std: 0.01337, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.78696, std: 0.02204, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: 0.78913, std: 0.02679, params: {'subsample': 0.9, 'colsample_bytree'

In [34]:
xgb1 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0.3, subsample=0.9, colsample_bytree=0.9, objective= 'rank:pairwise', nthread=4, scale_pos_weight=1, seed=27)
cross_val_score(xgb1,data[predictor],data['Loan_Status'],scoring='accuracy').mean()

0.76221106328710342

In [36]:
param_test5 = {'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]}
gsearch=GridSearchCV(estimator=XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'rank:pairwise',
 nthread=4,
 scale_pos_weight=1,
 seed=27),param_grid=param_test6,scoring='accuracy')
gsearch.fit(x_train,y_train)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

([mean: 0.78696, std: 0.02204, params: {'reg_alpha': 0},
  mean: 0.77609, std: 0.02412, params: {'reg_alpha': 0.001},
  mean: 0.78261, std: 0.01995, params: {'reg_alpha': 0.005},
  mean: 0.78913, std: 0.02679, params: {'reg_alpha': 0.01},
  mean: 0.78913, std: 0.01739, params: {'reg_alpha': 0.05}],
 {'reg_alpha': 0.01},
 0.78913043478260869)

In [89]:
xgb1 = XGBClassifier(learning_rate =0.001, n_estimators=1000, max_depth=1, min_child_weight=3, gamma=1, subsample=0.7, colsample_bytree=0.7, objective= 'rank:pairwise', nthread=4, scale_pos_weight=1, seed=27, reg_alpha=0.01)
cross_val_score(xgb1,data[predictor],data['Loan_Status'],scoring='accuracy').mean()

0.80948509485094855

In [61]:
data2=pd.read_csv('./train.csv')
def transform2(data):
    #data.Loan_Status=data.Loan_Status.map({'N':0,'Y':1})
    data.Gender=data.Gender.map({'Male':1,'Female':2})
    data.Married=data.Married.map({'Yes':1,'No':0})
    data.Education=data.Education.map({'Graduate':1,'Not Graduate':0})
    data.Self_Employed=data.Self_Employed.map({'No':0,'yes':1})
    data.Dependents=data.Dependents.map({'3+':3,'1':1,'2':2,'0':0})
    data.Property_Area=data.Property_Area.map({'Urban':1,'Semiurban':2,'Rural':3})
    return data
data2=transform2(data2)
loanid=data2['Loan_ID']
data2=data2.drop(['Loan_ID'],axis=1)
def myimputer(data):
    columns=data.columns
    imp=Imputer(missing_values='NaN',strategy='most_frequent',axis=0)
    data=imp.fit_transform(data)
    data=pd.DataFrame(data,columns=columns)
    return data
imputedata=myimputer(data2)
imputedata['income']=imputedata['ApplicantIncome']+imputedata['CoapplicantIncome']
imputedata=imputedata.drop(['ApplicantIncome','CoapplicantIncome'],axis=1)
xgb1.fit(data[predictor],data['Loan_Status'])
imputedatapred=xgb1.predict(imputedata)
final=pd.DataFrame({'Loan_ID':loanid,'Loan_Status':imputedatapred})
final.Loan_Status=final.Loan_Status.map({1:'Y',0:'N'})
final.to_csv('sample_submission.csv',mode='w',index=False)
final

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y
5,LP001054,Y
6,LP001055,Y
7,LP001056,N
8,LP001059,Y
9,LP001067,Y


In [7]:
import pandas as pd
data2=pd.read_csv('./train.csv')
def transform2(data):
    #data.Loan_Status=data.Loan_Status.map({'N':0,'Y':1})
    data.Gender=data.Gender.map({'Male':1,'Female':2})
    data.Married=data.Married.map({'Yes':1,'No':0})
    data.Education=data.Education.map({'Graduate':1,'Not Graduate':0})
    data.Self_Employed=data.Self_Employed.map({'No':0,'yes':1})
    data.Dependents=data.Dependents.map({'3+':3,'1':1,'2':2,'0':0})
    data.Property_Area=data.Property_Area.map({'Urban':1,'Semiurban':2,'Rural':3})
    return data
data2=transform2(data2)

data2.corr()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Gender,1.000000,-0.369612,-0.175970,0.049258,NaN,-0.053989,-0.083946,-0.106947,0.075117,-0.016337,-0.024556
Married,-0.369612,1.000000,0.343417,-0.014223,NaN,0.051332,0.077770,0.149519,-0.103810,0.004381,-0.002918
Dependents,-0.175970,0.343417,1.000000,-0.059161,NaN,0.118679,0.027259,0.163997,-0.100484,-0.050082,-0.006828
Education,0.049258,-0.014223,-0.059161,1.000000,NaN,0.140760,0.062290,0.171133,0.078784,0.081822,-0.065243
Self_Employed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ApplicantIncome,-0.053989,0.051332,0.118679,0.140760,NaN,1.000000,-0.116605,0.570909,-0.045306,-0.014715,0.009500
CoapplicantIncome,-0.083946,0.077770,0.027259,0.062290,NaN,-0.116605,1.000000,0.188619,-0.059878,-0.002056,-0.010522
LoanAmount,-0.106947,0.149519,0.163997,0.171133,NaN,0.570909,0.188619,1.000000,0.039447,-0.008433,0.045792
Loan_Amount_Term,0.075117,-0.103810,-0.100484,0.078784,NaN,-0.045306,-0.059878,0.039447,1.000000,0.001470,0.078748
Credit_History,-0.016337,0.004381,-0.050082,0.081822,NaN,-0.014715,-0.002056,-0.008433,0.001470,1.000000,0.001969
